In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [5]:
import dipy.reconst.sfm as sfm
import dipy.reconst.csdeconv as csd
import dipy.reconst.dti as dti
import dipy.core.gradients as grad
import dipy.data as dpd

In [6]:
import utils
from model import Model, BiExponentialIsotropicModel, my_responses

In [7]:
data = utils.read_data()

In [8]:
train_bvals = data['seen']['bvals']
train_bvecs = data['seen']['bvecs']
train_delta = data['seen']['delta']
train_Delta = data['seen']['Delta']
train_te = data['seen']['TE']
train_g = data['seen']['g']

In [9]:
test_bvals = data['unseen']['bvals']
test_bvecs = data['unseen']['bvecs']
test_delta = data['unseen']['delta']
test_Delta = data['unseen']['Delta']
test_te = data['unseen']['TE']
test_g = data['unseen']['g']

In [10]:
train_gtab = grad.gradient_table(train_bvals, train_bvecs, big_delta=train_Delta, small_delta=train_delta)
test_gtab = grad.gradient_table(test_bvals, test_bvecs, big_delta=test_Delta, small_delta=test_delta)

In [14]:
import sklearn.linear_model as lm
model1 = Model(train_gtab, isotropic=BiExponentialIsotropicModel,
               solver=lm.ElasticNet(l1_ratio=0.1, alpha=1e-5, positive=True, warm_start=True, max_iter=25000, 
                                                                               fit_intercept=True, normalize=True))

model2 = Model(test_gtab, isotropic=BiExponentialIsotropicModel,
               solver=lm.ElasticNet(l1_ratio=0.1, alpha=1e-5, positive=True, warm_start=True, max_iter=25000, 
                                                                fit_intercept=True, normalize=True))

In [21]:
all_predictions = []
for vox_idx in range(6):
    train_data = data['seen']['signal'][:, vox_idx]
    fit1 = model1.fit(train_data, train_te, train_g)
    predict1 = fit1.predict(test_gtab, test_te)
    all_predictions.append(predict1)
    predict2 = fit1.predict(train_gtab, train_te)
    plt.figure()
    plt.plot(train_gtab.bvals, train_data, '.')
    plt.plot(train_gtab.bvals, predict2, '.')
    plt.figure()
    plt.plot(test_gtab.bvals, predict1, '.')
    print(utils.LSE(predict2, train_data))

2.1194082528119882

In [35]:
np.savetxt('unseenSignal.txt', all_predictions, fmt='%.4f', header='%—voxel1——-voxel2——-voxel3-—-voxel4——-voxel5——-voxel6')